<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates Restrict S3 Buckets with READ/WRITE Permissions using unSkript legos.</b>
</div>

<br>

<center><h2>Restrict S3 Buckets with READ/WRITE Permissions to all Authenticated Users </h2></center>

# Steps Overview
 1) List all the S3 buckets.
 2) Filter buckets which has ACL public READ/WRITE permissions.
 3) Change the ACL Public READ/WRITE permissions to private.
     

Here we will use unSkript List of S3 Buckets Lego. This lego takes region: str as input. This inputs is used to list all the S3 buckets available.

In [21]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_get_s3_buckets_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_s3_buckets(handle, region: str) -> List:
    """aws_get_s3_buckets List all the S3 buckets.

          :type region: string
          :param region: location of the bucket

          :rtype: List all the S3 buckets
      """
    # connect to the S3 using resource
    s3Session = handle.resource("s3", region_name=region)

    # Get all the S3 Buckets
    response = s3Session.buckets.all()
    result = []
    for bucket in response:
        result.append(bucket.name)
    return result



task = Task(Workflow())
task.configure(credentialsJson='''{
}''')
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(outputName="bucketlist")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_s3_buckets, lego_printer=aws_get_s3_buckets_printer, hdl=hdl, args=args)

Here we will use unSkript Get Public S3 Bucket List Lego. This lego takes Bucket_List: list, Permission: str, region: str as input. This inputs is used to list S3 buckets which has public ACL permissions.

In [25]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_get_public_s3_buckets_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_public_s3_buckets(handle, Bucket_List: list, Permission: str, region: str) -> List:
    """aws_get_public_s3_buckets get list of public buckets.

          :type Bucket_List: list
          :param Bucket_List: list of S3 buckets.

          :type region: string
          :param region: location of the bucket.

          :rtype: Dict with the response info.
      """
    # connect to the S3 using client
    s3Client = handle.client('s3',
                             region_name=region)
    public_check = ["http://acs.amazonaws.com/groups/global/AuthenticatedUsers",
                   "http://acs.amazonaws.com/groups/global/AllUsers"]
    # filter public S3 buckets
    result = []
    for bucket in Bucket_List:
        try:
            res = s3Client.get_bucket_acl(Bucket=bucket)
            for grant in res["Grants"]:
                if 'Permission' in grant.keys() and Permission == grant["Permission"]:
                    if 'URI' in grant["Grantee"] and grant["Grantee"]["URI"] in public_check:
                        result.append(bucket)
        except Exception as e:
            continue

    return result

    
task = Task(Workflow())
task.configure(credentialsJson='''{
}''')
task.configure(inputParamsJson='''{
    "Bucket_List": "bucketlist",
    "Permission": "\\"READ\\"",
    "region": "\\"us-west-2\\""
    }''')
task.configure(outputName="PublicBucket")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_public_s3_buckets, lego_printer=aws_get_public_s3_buckets_printer, hdl=hdl, args=args)

Here we will use unSkript Change ACL Permissions of AWS Bucket Lego. This lego takes name: str, acl: str, region: str as input. This inputs is used to change the public ACL permissions to private.

In [24]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Dict
import pprint

from beartype import beartype

@beartype
def aws_put_bucket_acl_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_put_bucket_acl(handle, name: str, acl: str, region: str = None) -> Dict:
    """ aws_put_bucket_acl get Dict of buckets ACL change info.

            :type handle: Session
            :param handle: S3 boto3 client

            :type handle: str
            :param bucket: S3 bucket name where to set ACL on

            :type handle: str
            :param acl: canned ACL type - 'private'|'public-read'|'public-read-write'|'authenticated-read'.

            :rtype: nothing
    """
    # connect to the S3 using client
    s3Client = handle.client('s3',
                             region_name=region)

    # Put bucket ACL for the permissions grant
    response = s3Client.put_bucket_acl(
                    Bucket=name,
                    ACL=acl )

    return response


task = Task(Workflow())
task.configure(credentialsJson='''{
}''')
task.configure(inputParamsJson='''{
    "acl": "\\"public-read\\"",
    "name": "\\"testrunbook\\"",
    "region": "\\"us-west-2\\""
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_put_bucket_acl, lego_printer=aws_put_bucket_acl_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to Restrict S3 Buckets with READ/WRITE Permissions to all Authenticated Users. To view the full platform capabilities of unSkript please visit https://unskript.com